In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [2]:
# Path: Dataset_Train.ipynb
# Define the directory where the dataset is saved
dataset_dir = "C:/Users/91866/OneDrive - Amrita Vishwa Vidyapeetham/Desktop/git/live_facial_recognition/face_recognization/dataset"

# Initialize lists for the features and labels
X = []
y = []

# Loop through each subfolder in the dataset folder
for subfolder in os.listdir(dataset_dir):
    # Loop through each image in the subfolder
    for filename in os.listdir(os.path.join(dataset_dir, subfolder)):
        # Load the image
        img = cv2.imread(os.path.join(dataset_dir, subfolder, filename), cv2.IMREAD_GRAYSCALE)
        
        # Convert the image to a 1D numpy array
        img = img.flatten()
        
        # Append the image to the X list
        X.append(img)
        
        # Append the class label to the y list
        y.append(subfolder)


# Convert the X and y lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

# Evaluate the model on the test data
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))



Accuracy: 1.0
Confusion Matrix:
[[1 0 0]
 [0 4 0]
 [0 0 3]]
Classification Report:
              precision    recall  f1-score   support

         bot       1.00      1.00      1.00         1
        guru       1.00      1.00      1.00         4
     praveen       1.00      1.00      1.00         3

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



In [3]:
import cv2
import os
import pickle
import numpy as np
from mtcnn.mtcnn import MTCNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Define the directory where the dataset is saved
dataset_dir = "C:/Users/91866/OneDrive - Amrita Vishwa Vidyapeetham/Desktop/git/live_facial_recognition/face_recognization/dataset"

# Initialize the face detector
detector = MTCNN()

# Initialize the list of detected faces and their corresponding names
faces = []
names = []

# Loop through each subfolder in the dataset folder
for subfolder in os.listdir(dataset_dir):
    # Loop through each image in the subfolder
    for image_filename in os.listdir(os.path.join(dataset_dir, subfolder)):
        # Load the image
        filename = os.path.join(dataset_dir, subfolder, image_filename)
        img = cv2.imread(filename)
        
        # Check if the image exists and is not empty
        if img is not None:
            # Convert the image from BGR color (which OpenCV uses) to RGB color
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Detect faces in the image
            results = detector.detect_faces(rgb_img)
            
            # Loop through each detected face
            for result in results:
                # Extract the bounding box coordinates of the face
                x, y, w, h = result['box']
                
                # Extract the face region from the image
                face = rgb_img[y:y+h, x:x+w]
                
                # Resize the face image to a fixed size (e.g., 160x160)
                face = cv2.resize(face, (160, 160))
                
                # Convert the image from RGB color to grayscale
                face = cv2.cvtColor(face, cv2.COLOR_RGB2GRAY)
                
                # Convert the face image to a 1D numpy array
                face = face.flatten()
                
                # Append the face to the list of detected faces
                faces.append(face)
                
                # Append an associated label to the list of labels
                names.append(subfolder)

# Convert the faces and labels lists to NumPy arrays
faces = np.array(faces)
names = np.array(names)

# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100)
model.fit(faces, names)

# Save the trained model to a file
model_filename = "model.pkl"
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)

# Initialize a LabelEncoder
encoder = LabelEncoder()

# Encode the names
encoded_names = encoder.fit_transform(names)

# Save the label encoder to a file
encoder_filename = "encoder.pkl"
with open(encoder_filename, 'wb') as f:
    pickle.dump(encoder, f)

# Initialize the webcam (change to 0 if you're using the default camera)
cap = cv2.VideoCapture(0)

# Define the face detector
detector = MTCNN()

# Load the trained model and label encoder
model_filename = "model.pkl"
with open(model_filename, 'rb') as f:
    model = pickle.load(f)

encoder_filename = "encoder.pkl"
with open(encoder_filename, 'rb') as f:
    encoder = pickle.load(f)

# Loop until the user presses the Esc key
while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not capture an image.")
        break

    # Convert the frame from BGR color (which OpenCV uses) to RGB color
    rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the image
    results = detector.detect_faces(rgb_img)

    # Loop through each detected face
    for result in results:
        # Extract the bounding box coordinates of the face
        x, y, w, h = result['box']

        # Extract the face region from the image
        face = rgb_img[y:y+h, x:x+w]

        # Resize the face image to a fixed size (e.g., 160x160)
        face = cv2.resize(face, (160, 160))

        # Convert the image from RGB color to grayscale
        face = cv2.cvtColor(face, cv2.COLOR_RGB2GRAY)

        # Convert the face image to a 1D numpy array
        face = face.flatten()

        # Get a prediction for the person's name
        name = model.predict([face])

        # Check if the predicted name is in the encoder's classes
        if name in encoder.classes_:
            decoded_name = encoder.inverse_transform([name])
        else:
            decoded_name = "Unknown"  # Set a default label for unrecognized faces

        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 2)

        # Draw a label with the name below the face
        cv2.rectangle(frame, (x, y+h), (x+w, y+h+30), (255, 255, 255), cv2.FILLED)
        cv2.putText(frame, decoded_name[0], (x, y+h+20), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 0), 1)

    # Display the image with the bounding boxes and labels
    cv2.imshow("Face Recognition", frame)

    # Wait for the user to press the Esc key
    c = cv2.waitKey(1)
    if c == 27:
        break

# Release the camera
cap.release()


1/1 [==============================] - 0s 22ms/step
